In [1]:
import pandas as pd
import numpy as np
import seaborn as sns
import matplotlib.pyplot as plt

In [2]:
df = pd.read_csv("startup_funding.csv")

In [3]:
#See sample data
df.head()

,Industry,Date,StartupName,IndustryVertical,SubVertical,CityLocation,InvestorsName,InvestmentType,AmountInUSD,Remarks
0,Industry,1/8/2017,TouchKin,Technology,Predictive Care Platform,Bangalore,Kae Capital,Private Equity,"1,300,000",NaN
1,Industry,2/8/2017,Ethinos,Technology,Digital Marketing Agency,Mumbai,Triton Investment Advisors,Private Equity,NaN,NaN
2,Industry,2/8/2017,Leverage Edu,Consumer Internet,Online platform for Higher Education Services,New Delhi,"Kashyap Deorah, Anand Sankeshwar, Deepak Jain,...",Seed Funding,NaN,NaN
3,Industry,2/8/2017,Zepo,Consumer Internet,DIY Ecommerce platform,Mumbai,"Kunal Shah, LetsVenture, Anupam Mittal, Hetal ...",Seed Funding,"500,000",NaN
4,Industry,2/8/2017,Click2Clinic,Consumer Internet,healthcare service aggregator,Hyderabad,"Narottam Thudi, Shireesh Palle",Seed Funding,"850,000",NaN


In [4]:
#Drop useless columns
df.drop(columns =["Remarks"], inplace=True)

In [5]:
# See basic info of dataset
df.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 2372 entries, 0 to 2371
Data columns (total 9 columns):
Industry            2372 non-null object
Date                2372 non-null object
StartupName         2372 non-null object
IndustryVertical    2201 non-null object
SubVertical         1436 non-null object
CityLocation        2193 non-null object
InvestorsName       2364 non-null object
InvestmentType      2371 non-null object
AmountInUSD         1525 non-null object
dtypes: object(9)
memory usage: 166.9+ KB


In [6]:
#Interesting... No numbers even though AmountinUSD should be numerical.
# Try converting AmountinUSD to numbers
df.AmountInUSD.head()

0    1,300,000
1          NaN
2          NaN
3      500,000
4      850,000
Name: AmountInUSD, dtype: object

In [7]:
def str_to_num(item):
    if (type(item) == str):
        return float(item.replace(",",""))
        

In [8]:
df.AmountInUSD = df.AmountInUSD.apply(str_to_num)

In [9]:
#Converted Amount to numerical data
df.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 2372 entries, 0 to 2371
Data columns (total 9 columns):
Industry            2372 non-null object
Date                2372 non-null object
StartupName         2372 non-null object
IndustryVertical    2201 non-null object
SubVertical         1436 non-null object
CityLocation        2193 non-null object
InvestorsName       2364 non-null object
InvestmentType      2371 non-null object
AmountInUSD         1525 non-null float64
dtypes: float64(1), object(8)
memory usage: 166.9+ KB


In [10]:
# See most popular cities for startups.
df.CityLocation.value_counts().head(10)

Bangalore    627
Mumbai       446
New Delhi    381
Gurgaon      240
Pune          84
Noida         78
Hyderabad     76
Chennai       66
Ahmedabad     35
Jaipur        25
Name: CityLocation, dtype: int64

In [11]:
# Unsurprisingly, Bangalore takes the cake.
df.InvestmentType.value_counts()


Seed Funding      1271
Private Equity    1066
SeedFunding         30
PrivateEquity        1
Crowd Funding        1
Debt Funding         1
Crowd funding        1
Name: InvestmentType, dtype: int64

In [12]:
# Repeated values with different spelling for Investment Types. Need to fix.
df.loc[df["InvestmentType"]=="SeedFunding", "InvestmentType"] = "Seed Funding"
df.loc[df["InvestmentType"] == "PrivateEquity", "InvestmentType"] = "Private Equity"
df.loc[df["InvestmentType"]=="Crowd funding", "InvestmentType"] = "Crowd Funding"

In [13]:
# Create Interactive bar chart for Investment types. On click interactive event:
#   Show firms with most funding with that specific investment type.
df.InvestmentType.value_counts()

Seed Funding      1301
Private Equity    1067
Crowd Funding        2
Debt Funding         1
Name: InvestmentType, dtype: int64

In [14]:
# Create a df with unique firm entries with total Funding - For most popular industries (circle packing data).
# After that, create most popular firms by total funding. Filter by type of investment and IndustryType.


In [15]:
# Describe df.
df.describe()

,AmountInUSD
count,1.525000e+03
mean,1.203107e+07
std,6.403118e+07
min,1.600000e+04
25%,3.700000e+05
50%,1.070000e+06
75%,6.000000e+06
max,1.400000e+09


In [16]:
df.quantile([.1,.95])

,AmountInUSD
0.10,150000.0
0.95,49000000.0


In [17]:
# First rename IndustryVertical.
df.IndustryVertical.value_counts()

Consumer Internet                                          772
Technology                                                 313
eCommerce                                                  171
ECommerce                                                   53
Healthcare                                                  30
Logistics                                                   24
Education                                                   20
Food & Beverage                                             19
Finance                                                      9
Others                                                       6
Online Food Delivery                                         5
Online Education Platform                                    5
Real Estate                                                  4
FMCG                                                         3
Logistics Tech                                               3
Food Delivery Platform                                 

In [18]:
df.loc[df.IndustryVertical.str.contains("commerce", case=False, na=False), "IndustryVertical"] = "eCommerce"
df.loc[df.IndustryVertical.str.contains("online", case=False, na=False), "IndustryVertical"] = "Consumer Internet"
df.loc[df.IndustryVertical.str.contains("food", case=False, na=False), "IndustryVertical"] = "Food & Beverage"
df.loc[df.IndustryVertical.str.contains("health", case=False, na=False), "IndustryVertical"] = "Healthcare"
df.loc[df.IndustryVertical.str.contains("education", case=False, na=False), "IndustryVertical"] = "Education"
df.loc[df.IndustryVertical.str.contains("finance|fund", case=False, na=False), "IndustryVertical"] = "Finance"
allowed_vals = ["Consumer Internet","Education", "Finance", "eCommerce", "Healthcare"]
df.loc[~df["IndustryVertical"].isin(allowed_vals), "IndustryVertical"] = "Others"
(df.IndustryVertical.value_counts())

Others               1085
Consumer Internet     920
eCommerce             265
Healthcare             56
Education              28
Finance                18
Name: IndustryVertical, dtype: int64

In [19]:
df.IndustryVertical.value_counts()

Others               1085
Consumer Internet     920
eCommerce             265
Healthcare             56
Education              28
Finance                18
Name: IndustryVertical, dtype: int64

In [20]:
circle_pack_df = df.loc[df["IndustryVertical"] != "Others"]

In [21]:
circle_pack_df.AmountInUSD = circle_pack_df.AmountInUSD.fillna(0)

D:\Anaconda\lib\site-packages\pandas\core\generic.py:5096: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: http://pandas.pydata.org/pandas-docs/stable/indexing.html#indexing-view-versus-copy
  self[name] = value


In [22]:
circle_pack_df.drop(columns=["SubVertical"], inplace=True)

D:\Anaconda\lib\site-packages\pandas\core\frame.py:3940: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame

See the caveats in the documentation: http://pandas.pydata.org/pandas-docs/stable/indexing.html#indexing-view-versus-copy
  errors=errors)


In [23]:
circle_pack_df = circle_pack_df.drop_duplicates(subset='StartupName', keep='first')

In [24]:
circle_pack_df["value"] = 1

In [25]:
# circle_pack_df.to_csv("circle_pack_data.csv", index=False)

In [26]:
df.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 2372 entries, 0 to 2371
Data columns (total 9 columns):
Industry            2372 non-null object
Date                2372 non-null object
StartupName         2372 non-null object
IndustryVertical    2372 non-null object
SubVertical         1436 non-null object
CityLocation        2193 non-null object
InvestorsName       2364 non-null object
InvestmentType      2371 non-null object
AmountInUSD         1525 non-null float64
dtypes: float64(1), object(8)
memory usage: 166.9+ KB


In [27]:
df.CityLocation.fillna(df.CityLocation.mode()[0], inplace=True)


In [28]:
df.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 2372 entries, 0 to 2371
Data columns (total 9 columns):
Industry            2372 non-null object
Date                2372 non-null object
StartupName         2372 non-null object
IndustryVertical    2372 non-null object
SubVertical         1436 non-null object
CityLocation        2372 non-null object
InvestorsName       2364 non-null object
InvestmentType      2371 non-null object
AmountInUSD         1525 non-null float64
dtypes: float64(1), object(8)
memory usage: 166.9+ KB


In [29]:
df.AmountInUSD.fillna(df.AmountInUSD.median(), inplace=True)
df.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 2372 entries, 0 to 2371
Data columns (total 9 columns):
Industry            2372 non-null object
Date                2372 non-null object
StartupName         2372 non-null object
IndustryVertical    2372 non-null object
SubVertical         1436 non-null object
CityLocation        2372 non-null object
InvestorsName       2364 non-null object
InvestmentType      2371 non-null object
AmountInUSD         2372 non-null float64
dtypes: float64(1), object(8)
memory usage: 166.9+ KB


In [30]:
df.isna().sum()

Industry              0
Date                  0
StartupName           0
IndustryVertical      0
SubVertical         936
CityLocation          0
InvestorsName         8
InvestmentType        1
AmountInUSD           0
dtype: int64

In [31]:
df.Date.head(50)

0       1/8/2017
1       2/8/2017
2       2/8/2017
3       2/8/2017
4       2/8/2017
5       1/7/2017
6       3/7/2017
7       4/7/2017
8       5/7/2017
9       5/7/2017
10      5/7/2017
11      6/7/2017
12      6/7/2017
13      7/7/2017
14      7/7/2017
15     10/7/2017
16     11/7/2017
17     11/7/2017
18     11/7/2017
19     12/7/2017
20     12/7/2017
21     12/7/2017
22     12/7/2017
23    14/07/2017
24    14/07/2017
25    17/07/2017
26    17/07/2017
27    18/07/2017
28    18/07/2017
29    18/07/2017
30    18/07/2017
31    19/07/2017
32    19/07/2017
33    19/07/2017
34    20/07/2017
35    21/07/2017
36    21/07/2017
37    21/07/2017
38    21/07/2017
39    25/07/2017
40    25/07/2017
41    26/07/2017
42    26/07/2017
43    26/07/2017
44    27/07/2017
45    28/07/2017
46    28/07/2017
47    28/07/2017
48      1/6/2017
49      2/6/2017
Name: Date, dtype: object

In [32]:
df["Funding_date"] = pd.to_datetime(df['Date'], format='%d/%m/%Y', errors="coerce")

In [33]:
mask = df.Funding_date.isnull()
df.loc[mask, 'Funding_date'] = pd.to_datetime(df[mask]['Date'], format='%d/%m.%Y',
                                             errors='coerce')

In [34]:
df.to_csv("time_series_visual_data.csv")

In [35]:
df.loc[df["Date"].str.contains("2017")].AmountInUSD.sum()

6015335500.0

In [36]:
df.loc[df["Date"].str.contains("2016")].AmountInUSD.sum()

4263578608.0

In [37]:
4263578608.0 < 6015335500.0


True

In [38]:
df.loc[df["Date"].str.contains("2015")].AmountInUSD.sum()

8974762368.0

In [46]:
df.loc[df["Date"].str.contains("2017")].InvestmentType.value_counts()

Private Equity    226
Seed Funding      216
Debt Funding        1
Name: InvestmentType, dtype: int64

In [53]:
df.loc[df["Date"].str.contains("2015")].info()

<class 'pandas.core.frame.DataFrame'>
Int64Index: 936 entries, 1436 to 2371
Data columns (total 10 columns):
Industry            936 non-null object
Date                936 non-null object
StartupName         936 non-null object
IndustryVertical    936 non-null object
SubVertical         0 non-null object
CityLocation        936 non-null object
InvestorsName       930 non-null object
InvestmentType      936 non-null object
AmountInUSD         936 non-null float64
Funding_date        935 non-null datetime64[ns]
dtypes: datetime64[ns](1), float64(1), object(8)
memory usage: 80.4+ KB


In [48]:
df.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 2372 entries, 0 to 2371
Data columns (total 10 columns):
Industry            2372 non-null object
Date                2372 non-null object
StartupName         2372 non-null object
IndustryVertical    2372 non-null object
SubVertical         1436 non-null object
CityLocation        2372 non-null object
InvestorsName       2364 non-null object
InvestmentType      2371 non-null object
AmountInUSD         2372 non-null float64
Funding_date        2371 non-null datetime64[ns]
dtypes: datetime64[ns](1), float64(1), object(8)
memory usage: 185.4+ KB
